In [ ]:
import pandas as pd

In [ ]:
#---------------------------------------------------
#
# Config *** TO UPDATE ***
#
#---------------------------------------------------

# *** TO UPDATE: change the team number with your own team number
TEAM_NUMBER = "TeamDemo"

# File with AOIS definitions 
AOIS_DEFINITION_FILE = "../raw-data/"+TEAM_NUMBER+"/aois definitions/aois.csv"

# This file should be automatically generated from "4. Mapping Fixations and Saccades to AOIs"
FIFXATIONS_SACCADES_WITH_AOIS_FILE = "data/fixationAndSaccadesDataWithAOIs.csv"

# This file should be automatically generated from "5. AOI Visits"
DWELLS_FILE = "data/dwells.csv"

In [ ]:
# Read aois definition
aois_df = pd.read_csv(AOIS_DEFINITION_FILE)

In [ ]:
# Display aois definition
display(aois_df)

In [ ]:
# get list of AOIs 
aois = aois_df["AOI"].tolist()
print(f'AOIs: {aois}')

In [ ]:
# Read fixation and saccades data with AOIs using pandas library
fixationAndSaccadeData = pd.read_csv(FIFXATIONS_SACCADES_WITH_AOIS_FILE)
# set display.max_columns to none, to show all the columns when using head()
pd.set_option('display.max_columns', None)

In [ ]:
# Preview fixationAndSaccadeData
fixationAndSaccadeData.head()

In [ ]:
#----------------------------------------------------------------------------------------
#
# 1. Fixation measures at stimulus and AOI levels
#
#----------------------------------------------------------------------------------------

In [ ]:
#filter out data where FixID is NaN
fixationData = fixationAndSaccadeData[fixationAndSaccadeData['FixID'].notnull()].copy(deep=True)

In [ ]:
# Preview fixationData
fixationData.head()

In [ ]:
allAOIFixationStats = None

# Metrics for each task at the level of the whole stimulus 
for task in fixationData['SourceStimuliName'].unique():
    
    taskFixationData = fixationData[fixationData['SourceStimuliName']==task].copy(deep=True)
    
    # Compute different aggregations for Fixation-based metrics
    stats = taskFixationData.agg({
                                # Number of Fixations (Numerosity measure, cf. documentation/ETmetrics.csv)
                                'FixID':['count'],
                                # Fixation Duration
                                    # Sum: Total Fixation Duration (Position measure, cf. documentation/ETmetrics.csv)
                                    # Mean: Average Fixation duration (Position measure, cf. documentation/ETmetrics.csv)
                                    # Min: Minimum Fixation duration (Position measure, cf. documentation/ETmetrics.csv)
                                    # Max: Maximum Fixation duration (Position measure, cf. documentation/ETmetrics.csv)
                                'Fixation Duration':['sum','mean','min','max']
                             }).unstack().to_frame().dropna().T

    # edit stats formatting (by flattening it) to enable concating with the upcoming mesures
    stats.columns = ['_'.join(x) for x in list(zip(stats.columns.get_level_values(0), stats.columns.get_level_values(1)))]
    # rename columns
    stats = stats.rename(columns={"FixID_count": "Number of Fixations", 
                                  "Fixation Duration_sum": "Total Fixation Duration",
                                  "Fixation Duration_mean": "Average Fixation Duration",
                                  "Fixation Duration_min": "Minimum Fixation Duration",
                                  "Fixation Duration_max": "Maximum Fixation Duration",})
    
    stats.insert(0, 'Task',task)
    stats.insert(1, 'AOI', "Whole Stimulus")

    # Concate
    allAOIFixationStats = pd.concat([allAOIFixationStats, stats], axis=0)
    
# Reset_index for allAOIFixationStats
allAOIFixationStats = allAOIFixationStats.reset_index(drop=True)

In [ ]:
# Display the metrics for all AOIs aggregated
display(allAOIFixationStats)

In [ ]:
# Metrics for each task at the level of individual AOIs
for task in fixationData['SourceStimuliName'].unique():
    
    taskFixationData = fixationData[fixationData['SourceStimuliName']==task].copy(deep=True)

    # Iterate through the different AOIS
    for aoi in aois:
        # Compute different aggregations for Fixation metrics 
        stats = taskFixationData[taskFixationData[aoi]==1].groupby(aoi).agg({
                                # Number of Fixations (Numerosity measure, cf. documentation/ETmetrics.csv)
                                'FixID':['count'],
                                # Fixation Duration
                                    # Sum: Total Fixation Duration (Position measure, cf. documentation/ETmetrics.csv)
                                    # Mean: Average Fixation duration (Position measure, cf. documentation/ETmetrics.csv)
                                    # Min: Minimum Fixation duration (Position measure, cf. documentation/ETmetrics.csv)
                                    # Max: Maximum Fixation duration (Position measure, cf. documentation/ETmetrics.csv)
                                'Fixation Duration':['sum','mean','min','max']
                             })

        # Flatten dataframe
        stats.columns= ['_'.join(x) for x in list(zip(stats.columns.get_level_values(0), stats.columns.get_level_values(1)))]

        # Rename columns
        stats = stats.rename(columns={
                                  aoi:"AOI",
                                  "FixID_count": "Number of Fixations", 
                                  "Fixation Duration_sum": "Total Fixation Duration",
                                  "Fixation Duration_mean": "Average Fixation Duration",
                                  "Fixation Duration_min": "Minimum Fixation Duration",
                                  "Fixation Duration_max": "Maximum Fixation Duration"})

        # Set the value of "Task" to task 
        stats["Task"] = task
        # Set the value of "AOI" to aoi    
        stats["AOI"] = aoi

        
        # Concate
        allAOIFixationStats = pd.concat([allAOIFixationStats, stats], axis=0)
    
# Reset_index for allAOIFixationStats
allAOIFixationStats = allAOIFixationStats.reset_index(drop=True)

In [ ]:
# Display the metrics for all aois individually and aggregated
display(allAOIFixationStats)

In [ ]:
# export allAOIFixationStats to csv 
allAOIFixationStats.to_csv("data/aoiFixationStats.csv", index=False)

In [ ]:
#----------------------------------------------------------------------------------------
#
# 2. Saccade measures at stimulus and AOI levels 
#
#----------------------------------------------------------------------------------------

In [ ]:
#filter out data where SacID is NaN
SaccadeData = fixationAndSaccadeData[fixationAndSaccadeData['SacID'].notnull()].copy(deep=True)

In [ ]:
# Preview Saccade data
SaccadeData.head()

In [ ]:
allAOISaccadeStats = None

# Metrics for each task at the level of the whole stimulus 
for task in SaccadeData['SourceStimuliName'].unique():
    
    taskSaccadeData = SaccadeData[SaccadeData['SourceStimuliName']==task].copy(deep=True)

    # Compute different aggregations for saccade metrics 
    stats = taskSaccadeData.agg({
                                # Number of Saccades (Numerosity measure, cf. documentation/ETmetrics.csv)
                                'SacID':['count'],
                                # Saccade Duration
                                 # Sum: Total Saccade Duration (movement measure, cf. documentation/ETmetrics.csv)
                                 # Mean: Average Saccade Duration (movement measure, cf. documentation/ETmetrics.csv)
                                'Saccade Duration':['sum','mean'], 
                                 # Saccade Amplitude
                                  # Sum: Total Saccade Amplitude (movement measure, cf. documentation/ETmetrics.csv)
                                  # Mean: Average Saccade Amplitude (movement measure, cf. documentation/ETmetrics.csv)                                
                                'Saccade Amplitude':['sum','mean'], 
                                 # Saccade Peak Velocity
                                  # Mean: Average Peak Velocity of Saccades (movement measure, cf. documentation/ETmetrics.csv)
                                'Saccade Peak Velocity':['mean'], 

                                # Other metrics
                                #'Saccade Peak Acceleration':['sum','mean','max','min'],  
                                #'Saccade Peak Deceleration':['sum','mean','max','min'], 
                                # 'Saccade Direction':['mean'],
                             }).unstack().to_frame().dropna().T

    # edit stats formatting (by flattening it) to enable concating with the upcoming mesures
    stats.columns = ['_'.join(x) for x in list(zip(stats.columns.get_level_values(0), stats.columns.get_level_values(1)))]

    # Rename columns
    stats = stats.rename(columns={"SacID_count": "Number of Saccades", 
                                  "Saccade Duration_sum": "Total Saccade Duration",
                                  "Saccade Duration_mean": "Average Saccade Duration",
                                  "Saccade Amplitude_sum": "Total Saccade Amplitude",
                                  "Saccade Amplitude_mean": "Average Saccade Amplitude",
                                  "Saccade Peak Velocity_mean": "Average Peak Velocity of Saccades",                             
                                 })

    stats.insert(0, 'Task', task)
    stats.insert(1, 'AOI', "Whole Stimulus")
    
    # Concate
    allAOISaccadeStats = pd.concat([allAOISaccadeStats, stats], axis=0)
    
# Reset_index for allAOISaccadeStats
allAOISaccadeStats = allAOISaccadeStats.reset_index(drop=True)

In [ ]:
# Display the metrics for all AOIs aggregated
display(allAOISaccadeStats)

In [ ]:
# Metrics for each task at the level of individual AOIs
for task in fixationData['SourceStimuliName'].unique():
    
    taskSaccadeData = SaccadeData[SaccadeData['SourceStimuliName']==task].copy(deep=True)

    # Iterate through the different AOIS
    for aoi in aois:
        # Compute different aggregations for different metrics 
        stats = taskSaccadeData[taskSaccadeData[aoi]==1].groupby(aoi).agg({
                                # Number of Saccades (Numerosity measure, cf. documentation/ETmetrics.csv)
                                'SacID':['count'],
                                # Saccade Duration
                                 # Sum: Total Saccade Duration (movement measure, cf. documentation/ETmetrics.csv)
                                 # Mean: Average Saccade Duration (movement measure, cf. documentation/ETmetrics.csv)
                                'Saccade Duration':['sum','mean'], 
                                 # Saccade Amplitude
                                  # Sum: Total Saccade Amplitude (movement measure, cf. documentation/ETmetrics.csv)
                                  # Mean: Average Saccade Amplitude (movement measure, cf. documentation/ETmetrics.csv)                                
                                'Saccade Amplitude':['sum','mean'], 
                                 # Saccade Peak Velocity
                                  # Mean: Average Peak Velocity of Saccades (movement measure, cf. documentation/ETmetrics.csv)
                                'Saccade Peak Velocity':['mean'], 

                                # Other metrics
                                #'Saccade Peak Acceleration':['sum','mean','max','min'],  
                                #'Saccade Peak Deceleration':['sum','mean','max','min'], 
                                # 'Saccade Direction':['mean'],
                             })

        # Flatten dataframe
        stats.columns= ['_'.join(x) for x in list(zip(stats.columns.get_level_values(0), stats.columns.get_level_values(1)))]

        # rename columns
        stats = stats.rename(columns={
                                  aoi:"AOI",
                                  "SacID_count": "Number of Saccades", 
                                  "Saccade Duration_sum": "Total Saccade Duration",
                                  "Saccade Duration_mean": "Average Saccade Duration",
                                  "Saccade Amplitude_sum": "Total Saccade Amplitude",
                                  "Saccade Amplitude_mean": "Average Saccade Amplitude",
                                  "Saccade Peak Velocity_mean": "Average Peak Velocity of Saccades",                             
                                 })

        # Set the value of "Task" to task
        stats["Task"] = task
        # Set the value of "AOI" to aoi     
        stats["AOI"] = aoi

        #concate
        allAOISaccadeStats = pd.concat([allAOISaccadeStats, stats], axis=0)
    
# reset_index for allAOISaccadeStats
allAOISaccadeStats = allAOISaccadeStats.reset_index(drop=True)

In [ ]:
# Display the metrics for all aois individually and aggregated
display(allAOISaccadeStats)

In [ ]:
# export allAOISaccadeStats to csv 
allAOISaccadeStats.to_csv("data/aoiSaccadeStats.csv", index=False)

In [ ]:
#----------------------------------------------------------------------------------------
#
# 3. Dwell measures 
#
#----------------------------------------------------------------------------------------

In [ ]:
# Read dwell data using pandas library
dwells = pd.read_csv(DWELLS_FILE)
# set display.max_columns to none, to show all the columns when using head()
pd.set_option('display.max_columns', None)

In [ ]:
# Preview dwells
dwells.head()

In [ ]:
# Compute different aggregations for dwell metrics 
dwellsStats = dwells.groupby(['VisitedAOI','SourceStimuliName']).agg({
                                                   #Dwell Time:
                                                    #Sum: Total Dwell Time (Position measure, cf. documentation/ETmetrics.csv)
                                                    #Mean: Average Dwell Time (Position measure, cf. documentation/ETmetrics.csv)
                                                    #Count: Number of Dwells (Numerosity measure, cf. documentation/ETmetrics.csv)
                                                   'Dwell Time':['sum','mean','count'], 
    
                                                   #Number of Fixations in Dwell:
                                                    #Mean: Average Number of Fixations in Dwell (Numerosity measure, cf. documentation/ETmetrics.csv)
                                                   'Number of Fixations in Dwell':['mean',],
    
                                                   #Number of Saccades in Dwell:
                                                    #Mean: Average Number of Saccades in Dwell (Numerosity measure, cf. documentation/ETmetrics.csv)
                                                    'Number of Saccades in Dwell':['mean'],
    
                                                  }).reset_index() 


# Flatten dataframe
dwellsStats.columns= ['_'.join(x) for x in list(zip(dwellsStats.columns.get_level_values(0), dwellsStats.columns.get_level_values(1)))]

# Rename columns
dwellsStats = dwellsStats.rename(columns={"Dwell Time_sum": "Total Dwell Time", 
                              "Dwell Time_mean": "Average Dwell Time",
                              "Dwell Time_count": "Number of Dwells",
                              "Number of Fixations in Dwell_mean": "Average Number of Fixations in Dwell",
                              "Number of Saccades in Dwell_mean": "Average Number of Saccades in Dwell",                        
                             })


In [ ]:
# Display dwellsStats
display(dwellsStats)

In [ ]:
# Export dwellsStats to csv 
dwellsStats.to_csv("data/dwellsStats.csv", index=False)